# OCBA-m

Source. Chen et al. (2008) 

Efficient Simulation Budget Allocation for Selecting an Optimal Subset


https://pubsonline.informs.org/doi/pdf/10.1287/ijoc.1080.0268

In [ ]:
import pandas as pd
import numpy as np
from numba import jit
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

## Utility funcs

In [ ]:
def load_system(file_name, system, reps, reps_available, delim=','):
    """
    Reads system data from a txt file (assumes comma delimited by default).  
    Assumes that each column represents a system.
    Returns a numpy array.  Each row is a system (single row); each col is a replication
    
    
    @file_name = name of file containing csv data
    @system = index of system in txt file.
    @reps = replications wanted
    @reps_available = total number of replications that has been simulated.
    @delim = delimiter of file.  Default = ',' for CSV.  

    """
    
    return np.genfromtxt(file_name, delimiter=delim, usecols = system, skip_footer = (reps_available - reps))

In [ ]:
def simulate(data, k, allocations):
    """
    Simulates the systems.  
    Each system is allocated a different budget of replications
    
    Returns list of numpy arrays
    
    @allocations = numpy array.  budget of replications for the k systems 
    """
    return [data[i][0:allocations[i]] for i in range(k)]
    

In [ ]:
def get_ranks(array):
    """
    Returns a numpy array containing ranks of numbers within a input numpy array
    e.g. [3, 2, 1] returns [2, 1, 0]
    e.g. [3, 2, 1, 4] return [2, 1, 0, 3]
        
    @array - numpy array (only tested with 1d)
        
    """
    temp = array.argsort()
    ranks = np.empty_like(temp)
    ranks[temp] = np.arange(len(array))
    return ranks

In [ ]:
def bootstrap(data, boots=1000):
    """
    Returns a numpy array containing the bootstrap resamples
    Useful for creating a large number of experimental datasets for testing R&S routines
    
    @data = numpy.array of systems to boostrap
    @boots = number of bootstrap (default = 1000)
    """

    experiments = boots
    designs = data.shape[0]
    samples = data.shape[1]

    datasets = np.zeros((experiments, designs, samples))
     
    for exp in range(experiments):
        
        for design in range(designs):

            for i in range(samples):

                datasets[exp][design][i] = data[design][round(np.random.uniform(0, samples)-1)]
      
    return datasets

In [ ]:
@jit(nopython=True)
def crn_bootstrap(data, boots=1000):
    """
    Returns a numpy array containing the bootstrap resamples
    Useful for creating a large number of experimental datasets for testing R&S routines
    
    @data = numpy.array of systems to boostrap
    @boots = number of bootstrap (default = 1000)
    """

    experiments = boots
    designs = data.shape[0]
    samples = data.shape[1]
    
    datasets = np.zeros((experiments, designs, samples))
     
    for exp in range(experiments):
        
         for i in range(samples):

                row = data.T[np.random.choice(data.shape[0])]
                
                for design in range(designs):
                    datasets[exp][design][i] = row[design]  
      
    return datasets

# Walk through of how Ocba-m works

##  Input

**k** = total number of designs

**m** = no. of top designs to select

**T** = total budget (replications)

**n_0** = l number of replications  (>=5)

**delta** - additional budget to allocate between each simulation  

*where T - k.n_0 is a multiple of delta*


### Full Implementation 

In [ ]:
@jit(nopython=True)
def summary_statistics(systems, allocations):
    means = np.array([array.mean() for array in systems])
    stds = np.array([array.std() for array in systems])
    ses = np.divide(stds, np.sqrt(allocations))
    
    return means, ses

In [ ]:
def top_m(means, m):
    """
    Returns the top m system designs
    Largest sample means
    """
    ranks = get_ranks(means)
    return np.argpartition(ranks, -m)[-m:]

In [ ]:
def bottom_m(means, m):
    """
    Returns the bottom m system designs
    Smallest sample means
    """
    ranks = get_ranks(means)
    return np.argpartition(ranks, m)[:m]

In [ ]:
@jit(nopython=True)
def parameter_c(means, ses, k, m):
    order = np.argsort(means)
    s_ses = ses[order]
    s_means = means[order]

    return((s_ses[k-m+1] * s_means[k-m]) + (s_ses[k-m] * s_means[k-m+1]))/(s_ses[k-m]+s_ses[k-m+1])  

In [ ]:
def ocba_m(dataset, k, allocations, T, delta, m):
    
    while allocations.sum() < T:
        
        #simulate systems using new allocation of budget
        reps = simulate(dataset, k, allocations) 
        
        #calculate sample means and standard errors
        means, ses = summary_statistics(reps, allocations)
        
        #calculate parameter c and deltas
        c = parameter_c(means, ses, k, m)
        deltas = means - c
        
        print(deltas)
        
        #allocate
        for i in range(delta):
            values = np.divide(allocations, np.square(np.divide(ses, deltas)))
            ranks = get_ranks(values)
            allocations[ranks.argmin()] += 1
            
    return means, ses, allocations

In [ ]:
def cs(selected_top_m, true_top_m):
    """
    Returns boolean value.  
    True = correct selection of top m
    False = incorrect selection (one or more of selected top m is incorrect)
    
    Keyword arguments:
    selected_top_m - numpy.array containing the indexes of the top m means selected by the algorithm
    true_top_m - numpy.array containing the indexes of the true top m means
    
    """
    return np.array_equal(np.sort(selected_top_m), true_top_m)

In [ ]:
def oc(true_means, selected_top_m, true_top_m):
    """
    Return the opportunity cost of the selection
    Method penalised particular bad solutions more than mildly bad
    
    @true_means - numpy.array of true means
    @selected_top_m - numpy.array containing the indexes of the top m means selected by the algorithm
    @true_top_m - numpy.array containing the indexes of the true top m means
    
    """
    return (true_means[selected_top_m] - true_means[true_top_m]).sum()

In [ ]:
def p_cs(correct_selections):
    """
    Returns the probability of correct selection P{cs}
    Keyword arguments:
    
    correct_selections - list indicating if a given budget found the top m e.g. [False, True, True]
    """
    return np.array(correct_selections).sum() / len(correct_selections)

In [ ]:
def p_cs2(correct_selections):
    """
    Returns the probability of correct selection P{cs}
    
    @correct_selections - numpy.array of budget versus exp. Contains True or False indicating correct selection.
    
    """
    return np.mean(correct_selections, axis=1)

In [ ]:
def e_oc(opportunity_costs):
    """
    Return the expected opportunity cost E{oc}
    
    @opportunity costs - arrange of opportunity cost per budget
    """
    return np.array(opportunity_costs).mean()

In [ ]:
def e_oc2(opportunity_costs):
    """
    Return the expected opportunity cost E{oc}
    
    @opportunity costs - arrange of opportunity cost per budget
    """
    return np.mean(opportunity_costs, axis=0)

# Numerical test - single experiment

### Create Experimental Data#

#### Use the following code to create independent samples

In [ ]:
def experiments_independent_samples(ifile_name, boots=1000):
    data = np.genfromtxt(ifile_name, delimiter=",", skip_footer=0).transpose()
    experiments = bootstrap(data, boots=boots)
    return experiments

#### Use the following code to create CRN data sets

In [ ]:
def experiments_dependent_samples(ifile_name, boots=1000):
    data = np.genfromtxt(ifile_name, delimiter=",", skip_footer=0).transpose()
    experiments = crn_bootstrap(data, boots=boots)
    return experiments

Use the `numerical_experiment()` function to run experiments of OBCA-m across multiple experimental datasets and budgets.

For example, you may have 100 datasets to test across budgets of 1000, 2000, 3000 and 4000 replications.

In [ ]:
def numerical_experiment(experiments, budgets, delta, true_top_m, n_0,
                         opt='max'):
    """
    Conduct a user set number of numerical experiments on the algorithm
    for different computational budgets
    
    Returns:
    1. numpy.array containing P{cs} for each budget
    2. numpy.array containing E{oc} for each budget
    
    Keyword arguments:
    experiments -- numpy.array[experiments][designs][replication]
    budgets -- python.list containing budgets
    delta - maximum amount of budget to allocate per run. 
    true_top_m -- top m (i.e. the systems that should be returned)
    n_0 -- initial number of replications per system
    
    """
    n_experiments = experiments.shape[0]
    k = experiments.shape[1]
    m = true_top_m.shape[0]
        
    correct_selections = np.zeros((n_experiments, len(budgets)))
    opportunity_costs = np.zeros((n_experiments, len(budgets)))

    for exp in range(n_experiments):

        for t in range(len(budgets)):

            allocations = np.full(k, n_0, dtype=int)
                        
            means, ses, allocations = ocba_m(experiments[exp], k, 
                                             allocations, budgets[t], 
                                             delta, m)
            
            
            if opt == 'max':
                selected_top_m = top_m(means, m)
                print('max')
            else:
                selected_top_m= bottom_m(means, m)

            correct_selections[exp][t] = cs(selected_top_m, true_top_m)
            print(selected_top_m)
            
    return correct_selections

# Experiments for CRN data

In [ ]:
def get_budgets(max_t, min_t, increment_t):
    #incremental budgets 200, 400, .... T
    budgets = [i for i in range(min_t, max_t + increment_t, increment_t)]
    return budgets

In [ ]:
def experiment_1():
    T = 2000
    increment_t = 100
    min_budget = 300
    n_0 = 20
    delta = 50

    #data file for this experiment.
    ifile_name = 'data/EG1a_CRN.csv'

    #info for correct selection
    true_top_m = np.array([7, 8, 9])

    #incremental budgets 200, 400, .... T
    budgets = [400] #get_budgets(T, min_budget, increment_t)
    
    #generate experimental dataset
    experiments = experiments_independent_samples(ifile_name, boots=10)
    
    #run numerical experiment
    css = numerical_experiment(experiments, budgets, delta, true_top_m, n_0)
    
    return css

In [ ]:
css = experiment_1()

In [ ]:
df = pd.DataFrame(css)
df.to_clipboard()

In [ ]:
df

## Experiments for Law Inventory Example

### Maximisation Example (Top 3)

In [ ]:
def experiment_2():
    T = 300
    increment_t = 100
    min_budget = 300
    n_0 = 20
    delta = 50
    n_experiments = 5

    #data file for this experiment
    ifile_name = 'data/EGLaw_CRN.csv'
    
    #info for correct selection (maximisation problem)
    true_top_m = np.array([6, 7, 8]) 
    
    #incremental budgets 200, 400, .... T
    budgets = get_budgets(T, min_budget, increment_t)
    
    #generate experimental dataset
    experiments = experiments_dependent_samples(ifile_name, boots=n_experiments)
    
    #run numerical experiment
    css = numerical_experiment(experiments, budgets, delta, true_top_m, n_0)
    
    return css

### Minimisation Example (Bottom 2)

In [ ]:
def experiment_3():
    T = 1000
    increment_t = 100
    min_budget = 300
    n_0 = 20
    delta = 50
    n_experiments = 10000

    #data file for this experiment
    ifile_name = 'data/EGLaw_CRN.csv'
    
    #info for correct selection (maximisation problem)
    true_top_m = np.array([1, 2]) 
    
    #incremental budgets 200, 400, .... T
    budgets = get_budgets(T, min_budget, increment_t)
    
    #generate experimental dataset
    experiments = experiments_dependent_samples(ifile_name, boots=n_experiments)
    
    #run numerical experiment
    css = numerical_experiment(experiments, budgets, delta, true_top_m, n_0, 
                               opt='min')
    
    return css

# Run Experiments

In [ ]:
results_law_max = experiment_2()

In [ ]:
pd.DataFrame(results_law_max)

In [ ]:
results_law_min = experiment_3()

In [ ]:
pd.DataFrame(results_law_min)

In [ ]:
#df = pd.DataFrame(css)
#df["select"] = top_m_list
#df["allocations"] = allocation_list
#df["means_list"] = np.around(means_list, decimals=4).tolist()
#df.head()

In [ ]:
#print("top m: {0}".format(df[0:1]["select"].tolist()))
#print("Means selected: {0}".format(df[0:1]["means_list"].tolist()))
#print("Allocations: {0}".format(df[0:1]["allocations"].tolist()))

In [ ]:
budgets = get_budgets(3000, 300, 100)
fig = plt.figure(figsize=(10,6))
ax = pd.DataFrame(css, columns=budgets).mean(axis=0).plot(style='.-', ms=15)
ax.set_ylabel("P{CS}")
ax.set_xlabel("Budget (max no. replications)")
ax.set_xticks(ticks = [i for i in range(200, 8000, 200)])

ax.plot()

In [ ]:
fig = plt.figure(figsize=(10,6))
ax = pd.DataFrame(ocs, columns=budgets).mean(axis=0).plot(style='.-', ms=15)
ax.set_ylabel("E{OC}")
ax.set_xlabel("Budget (max no. replications)")
ax.set_xticks(ticks = [i for i in range(200, 8000, 200)])

ax.plot()